In [57]:
command = """az account list --query "[?name=='AAFC VSE Benefit'].id" -o tsv"""
subscription_id = !{command}
subscription_id = subscription_id[0]
assert subscription_id, "No subscription id found."

In [58]:
from pathlib import Path
template_dir = Path("template")
assert template_dir.exists()

In [59]:
output_dir = Path("rendered")

In [60]:
if output_dir.exists():
    assert input("type yes to clear render dir") == "yes"
    import shutil
    shutil.rmtree(output_dir)

In [61]:
# ensure "rendered" is in gitignore exactly once
with open(".gitignore", "r") as f:
    gitignore = f.read()
    gitignore = gitignore.strip()
    gitignore = gitignore.split("\n")
    gitignore = [line.strip() for line in gitignore]
    gitignore = [line for line in gitignore if line]
    gitignore = set(gitignore)
    gitignore.add("rendered")
    gitignore = sorted(gitignore)
    newcontent = "\n".join(gitignore)
    newcontent += "\n"
    
    # write the new contents
with open(".gitignore", "w") as f:
    f.write(newcontent)

print(f"Ignoring {len(gitignore)} patterns.")

Ignoring 3 patterns.


In [62]:
warning_header = """
# THIS FILE IS MANAGED BY AUTOMATION!
# IF YOU WANT TO MAKE CHANGES, EDIT THE TEMPLATE AND RE-RUN THE AUTOMATION
# GO TO `iac-kubernetes/manifest_templates.ipynb` TO FIND THE AUTOMATION
""".strip()

In [63]:
variables = {
    "subscription_id": subscription_id, # useful for terraform boilerplate.tf files
    "warning_header": warning_header, # useful to avoid changing the output by mistake
}

In [64]:
%pip install -q jinja2 pathspec

Note: you may need to restart the kernel to use updated packages.


In [65]:
import pathspec
ignore = pathspec.PathSpec.from_lines(pathspec.patterns.GitWildMatchPattern, gitignore)

In [66]:
from jinja2 import Environment, StrictUndefined
jinja = Environment(undefined=StrictUndefined)

In [67]:
# perform the rendering
import shutil
    
# discover source template files
for template_file in template_dir.glob("**/*"):
    # if not a file
    if not template_file.is_file():
        print(f"Skipping {template_file} because it is not a file")
        continue
    # if not ignored
    if ignore.match_file(str(template_file)):
        print(f"Just copying {template_file} because it matches gitignore")
        output_path = output_dir / template_file.relative_to(template_dir)
        output_path.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy(template_file, output_path)
        continue
    # if not a template file
    perform_templating = template_file.suffix in [".yaml",".yml",".tf",".env"]
    perform_templating |= template_file.name[0] == "."
    if not perform_templating:
        print(f"Just copying {template_file} because it is not a configured extension")
        output_path = output_dir / template_file.relative_to(template_dir)
        output_path.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy(template_file, output_path)
        continue
    else:
    # perform templating
        print(f"Rendering {template_file} because it is a configured extension")
        template_content = template_file.read_text(encoding = "utf-8")

        template = jinja.from_string(template_content)
        content = template.render(**variables)
        
        print(f"Writing rendered {output_path}")
        output_path = output_dir / template_file.relative_to(template_dir)
        output_path.parent.mkdir(parents=True, exist_ok=True)
        with open(output_path, "w", encoding="utf-8") as f:
            f.write(content)

Rendering template\main.tf because it is a configured extension
Writing rendered rendered\terraform\.terraform\providers\registry.terraform.io\integrations\github\5.40.0\windows_amd64\terraform-provider-github_v5.40.0.exe


In [68]:
if output_dir.exists():
    %pushd {output_dir}
    !terraform fmt -recursive
    %popd

d:\Repos\Azure\ca.teamdman.iac\rendered
d:\Repos\Azure\ca.teamdman.iac
popd -> d:\Repos\Azure\ca.teamdman.iac
